In [34]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk import FreqDist, classify, NaiveBayesClassifier

In [35]:
import re, string, random, nltk

In [36]:
def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [37]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

In [38]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

In [39]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
neutral_tweets = twitter_samples.strings('tweets.20150430-223406.json')

In [40]:
stop_words = stopwords.words('english')

In [41]:
positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
neutral_tweet_tokens = twitter_samples.tokenized('tweets.20150430-223406.json')

In [42]:
positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
neutral_cleaned_tokens_list = []

In [43]:
for tokens in positive_tweet_tokens:
        positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
        negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
        
for tokens in neutral_tweet_tokens:
        neutral_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [44]:
all_pos_words = get_all_words(neutral_cleaned_tokens_list)

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[('rt', 13540), ('…', 7663), ('miliband', 5318), ('snp', 4611), ('tories', 3953), ('ed', 2812), ('labour', 2630), ('#bbcqt', 2617), ('cameron', 2468), ('say', 2223)]


In [45]:
positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
neutral_tokens_for_model = get_tweets_for_model(neutral_cleaned_tokens_list)

In [46]:
positive_dataset = [(tweet_dict, "Positive")
                         for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                         for tweet_dict in negative_tokens_for_model]

neutral_dataset = [(tweet_dict, "Neutral")
                         for tweet_dict in neutral_tokens_for_model]

In [47]:
dataset = positive_dataset + negative_dataset + neutral_dataset

In [48]:
 random.shuffle(dataset)

In [49]:
print(len(dataset))

30000


In [50]:
print(len(positive_dataset))
print(len(negative_dataset))
print(len(neutral_dataset))

5000
5000
20000


In [63]:
train_data = dataset[:25000]
test_data = dataset[5000:]

In [64]:
classifier = nltk.NaiveBayesClassifier.train(train_data)

In [65]:
print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.99604


In [56]:
print(classifier.show_most_informative_features(10))

Most Informative Features
                      :( = True           Negati : Neutra =   3527.6 : 1.0
                      :) = True           Positi : Negati =   1199.4 : 1.0
                      rt = True           Neutra : Negati =   1120.2 : 1.0
                      :d = True           Positi : Neutra =    842.7 : 1.0
                     :-( = True           Negati : Neutra =    625.2 : 1.0
                  labour = True           Neutra : Positi =    203.0 : 1.0
                     :-) = True           Positi : Neutra =    150.1 : 1.0
                   wanna = True           Negati : Neutra =    143.5 : 1.0
                   david = True           Neutra : Negati =    142.4 : 1.0
                      ed = True           Neutra : Negati =    135.1 : 1.0
None


In [57]:
custom_tweet = "i'm sad"

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative


In [58]:
import pickle

In [59]:
Pkl_Filename = "Pickle_RL_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(classifier, file)

In [60]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Pickled_LR_Model = pickle.load(file)

Pickled_LR_Model

In [61]:
print("Accuracy is:", classify.accuracy(classifier, test_data))

Accuracy is: 0.9917333333333334


In [62]:
custom_tweet = "Jackson had never been so happy, and at ease, and she loved Connor with all her heart, yet she was a little sad, feeling they were growing apart."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Negative
